In [ ]:
from flask import Flask, jsonify

# Create an app
app = Flask(__name__)

# Define routes
@app.route("/")
def home():
    return (
        f"Welcome to the Hawaii Climate Analysis API!<br/>"
        f"Available Routes:<br/>"
        f"/api/v1.0/precipitation<br/>"
        f"/api/v1.0/stations<br/>"
        f"/api/v1.0/tobs<br/>"
        f"/api/v1.0/&lt;start&gt;<br/>"
        f"/api/v1.0/&lt;start&gt;/&lt;end&gt;<br/>"
    )

@app.route("/api/v1.0/precipitation")
def precipitation():
    # Query to retrieve the last 12 months of precipitation data
    prcp_data = session.query(Measurement.date, Measurement.prcp).\
                filter(Measurement.date >= one_year_ago).all()

    # Convert the query results to a dictionary
    prcp_dict = {date: prcp for date, prcp in prcp_data}

    return jsonify(prcp_dict)

@app.route("/api/v1.0/stations")
def stations():
    # Query to retrieve the list of stations
    stations = session.query(Station.station).all()

    # Convert the query results to a list
    station_list = list(np.ravel(stations))

    return jsonify(station_list)

@app.route("/api/v1.0/tobs")
def tobs():
    # Query to retrieve the previous 12 months of temperature observation data for the most active station
    tobs_data = session.query(Measurement.date, Measurement.tobs).\
                filter(Measurement.station == most_active_station).\
                filter(Measurement.date >= one_year_ago).all()

    # Convert the query results to a list of dictionaries
    tobs_list = [{date: tobs} for date, tobs in tobs_data]

    return jsonify(tobs_list)

@app.route("/api/v1.0/<start>")
def temp_start(start):
    # Query to calculate TMIN, TAVG, and TMAX for all dates greater than or equal to the start date
    temps = session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
            filter(Measurement.date >= start).all()

    # Convert the query results to a list of dictionaries
    temp_list = [{'TMIN': temps[0][0], 'TAVG': temps[0][1], 'TMAX': temps[0][2]}]

    return jsonify(temp_list)

@app.route("/api/v1.0/<start>/<end>")
def temp_start_end(start, end):
    # Query to calculate TMIN, TAVG, and TMAX for dates between the start and end date (inclusive)
    temps = session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
            filter(Measurement.date >= start).filter(Measurement.date <= end).all()

    # Convert the query results to a list of dictionaries
    temp_list = [{'TMIN': temps[0][0], 'TAVG': temps[0][1], 'TMAX': temps[0][2]}]

    return jsonify(temp_list)

if __name__ == '__main__':
    app.run(debug=True)